In [ ]:
import plotly.express as px
import pandas as pd
import pymssql
import json

In [ ]:
database = "arctic_analysts_capstone"
user = "arctic_analysts"
password  = "ThisPassw0rd!"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

def get_data(n = False):
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f"SELECT DISTINCT Date FROM building_permits"

    available_years_df = pd.read_sql(query, conn)
    available_years_df.sort_values(by = 'Date', inplace = True)
    available_dates = available_years_df.Date.astype('str').tolist()
    
    picked_date = available_dates[n]
    columns_to_select = 'FIPS, County, NewUnits'
    query = f"SELECT {columns_to_select} FROM building_permits WHERE building_permits.Date = '{picked_date}'"
    bpm_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {available_years_df.shape[0]}')

    return bpm_df

In [ ]:
n = 0
while n < 300:
    bpm_df = get_data(n)
    print(bpm_df['NewUnits'].sum())
    n += 1

In [ ]:
bpm_df = bpm_df[["CountyFips", "County", "StateFips", "1_Unit"]].copy()

In [ ]:
bpm_df

In [ ]:
bpm_df['locale_fips'] = bpm_df.StateFips + bpm_df.CountyFips
bpm_df

In [ ]:
with open('2021_US_Counties_3.7.json') as file:
    counties = json.load(file)

In [ ]:
fig = px.choropleth(bpm_df, geojson=counties, locations = 'locale_fips', color = '1_Unit',
                   color_continuous_scale = "rdylgn_r", scope = "usa")

In [ ]:
with open("test_map.html", 'w') as mapfile:
    mapfile.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
mapfile.close()